
To run this notebook, 
1. put this notebook inside the `CAS_Cropping` folder of the `planted_solutions` repository 
2. put `fcidump.2_co2_6-311++G__` in the folder `fcidumps_catalysts`
3. Using Python 3.10 (3.9 may work), run the following commands to install packages:
    ```
    python -m pip install openfermion
    python -m pip install h5py
    python -m pip install notebook
    python -m pip install git+https://github.com/jtcantin/dmrghandler
    ```

### Workflow
Issues highlighted in <font color="red">**RED**</font>
1. Load FCIDUMP file for a catalyst
    - Hamiltonian assumed is `H = E_0 + h_ij a†_i a_j + 0.5*g_ijkl a†_i a†_k a_l a_j`, `ijkl` refer to *spatial* orbitals
2. Combine one body tensor `h_ij` with two body tensor `g_ijkl` to get combined two body tensor `Γ_pqrs,στ`, where `σ=α,β; τ=α,β`
    - Hamiltonian assumed is `H = 0.5*Γ_pqrs,στ a†_pσ a_qσ a†_rτ a_sτ = Γ_PQRS a†_P a_Q a†_R a_S`, `PQRS` refer to *spin* orbitals
3. Symmetrize combined two body tensor `γ_pqrs,στ = 0.5*(Γ_pqrs,αα + Γ_pqrs,ββ + Γ_pqrs,αβ + Γ_pqrs,βα) / 4`
    - This symmetrization is done as all the Zapata catalyst Hamiltonians have this symmetry and the final FCIDUMP files need this symmetry 
4. Obtain CAS-cropped planted solution two body tensor `tbt_1`
    - αβ symmetry maintained: `tbt_1_pqrs,αα = tbt_1_pqrs,ββ = tbt_1_pqrs,αβ = tbt_1_pqrs,βα`
5. Add balance operators (controls number of electrons in each block) to get `tbt_2`
    - αβ symmetry <font color="red">**BROKEN**</font> -> problem as the catalyst Hamiltonians respect this symmetry
6. Generate killer operator `K`
7. Save planted solution to `.pkl` file
8. Use the function `construct_Hamiltonian_with_solution` to load `.pkl` file and produce:
    - `tbt_1`: original CAS-cropped planted solution two body tensor
        - <font color="red">**NOTE**</font>: balance operators NOT included
    - `tbt_1_hidden`: `tbt_1` plus hiding rotations 
        - <font color="red">**THESE ROTATIONS MIX SPIN-ORBITALS**</font> -> problem as this is unphysical
    - `tbt_3`: `tbt_1` with killer operators
        - αβ symmetry <font color="red">**BROKEN**</font>  -> problem as the catalyst Hamiltonians respect this symmetry
    - `tbt_3_hidden`: `tbt_3` plus hiding rotations 
        - <font color="red">**THESE ROTATIONS MIX SPIN-ORBITALS**</font> -> problem as this is unphysical
    - <font color="red">**NOTE**</font>: `tbt_2` is not returned by `construct_Hamiltonian_with_solution`
9. Convert tensors from spin orbitals to spatial orbitals
    - New Hamiltonian assumed is `H = H_ij a†_i a_j + 0.5*G_ijkl a†_i a†_k a_l a_j`, `ijkl` refer to *spatial* orbitals
        - This is necessary for the FCIDUMP file
10. Save `H_ij` and `G_ijkl` to FCIDUMP file
11. Load FCIDUMP file
12. Run DMRG in 'SU(2)' symmetry mode
    - This is the same mode as used for the catalyst Hamiltonians. It needs the αβ symmetry.

### Below code run is for the catalyst from the file `fcidump.2_co2_6-311++G__`, with a single block

In [25]:
### Parameters
tol = 1e-5
tol2 = 1e-5
balance_strength = 2
save = False
method_name = 'CAS-Cropping'
# Number of spatial orbitals in a block
block_size = 3 #6 to make a single block for testing
# Number of electrons per block
ne_per_block = 4 #8 to make a single block for testing
# +- difference in number of electrons per block
ne_range = 0
# Number of killer operators for each CAS block
n_killer = 3
# Running Full CI to check compute the ground state, takes exponentially amount of time to execute
FCI = True
# Checking symmetries of the planted Hamiltonian, very costly
check_symmetry = True
# File path and name
fcidump_path ="fcidumps_catalysts"
fcidump_filename = "fcidump.2_co2_6-311++G__"
fcidump_output_path = "fcidumps_catalysts_planted_solutions"

In [26]:
"""Construct CAS Hamiltonians with cropping functions
"""
import saveload_utils as sl
import ferm_utils as feru
import csa_utils as csau
import var_utils as varu
import openfermion as of
import numpy as np
from sdstate import *
from itertools import product
import random
import h5py
import sys
from matrix_utils import construct_random_sz_unitary
import pickle
import os
from pathlib import Path
import CAS.dmrghandler.src.dmrghandler.pyscf_wrappers
import CAS.dmrghandler.src.dmrghandler.dmrg_calc_prepare
import CAS.dmrghandler.src.dmrghandler.qchem_dmrg_calc
import CAS.dmrghandler.src.dmrghandler as dmrghandler
import pyscf.tools.fcidump



def construct_blocks(b: int, spin_orbs: int):
    """Construct CAS blocks of size b * 2 for spin_orbs number of orbitals"""
    b = b * 2
    k = []
    tmp = [0]
    for i in range(1, spin_orbs):
        if i % b == 0:
            k.append(tmp)
            tmp = [i]
        else:
            tmp.append(i)
    if len(tmp) != 0:
        k.append(tmp)
    return k
        
def get_truncated_cas_tbt(Htbt, k, casnum):
#     Trunctate the original Hamiltonian two body tensor into the cas block structures
    cas_tbt = np.zeros(Htbt.shape)
    cas_x = np.zeros(casnum)
    idx = 0
    for block in k:
        for a in block:
            for b in block:
                for c in block:
                    for d in block:
                        cas_tbt [a,b,c,d] = Htbt [a,b,c,d]
                        cas_x[idx] = Htbt[a,b,c,d]
                        idx += 1
    return cas_tbt, cas_x

def in_orbs(term, orbs):
    """Return if the term is a local excitation operator within orbs"""
    if len(term) == 2:
        return term[0][0] in orbs and term[1][0] in orbs
    elif len(term) == 4:
        return term[0][0] in orbs and term[1][0] in orbs and term[2][0] in orbs and term[3][0] in orbs
    return False

def transform_orbs(term, orbs):
    """Transform the operator term to align the orbs starting from 0"""
#     pass
    if len(term) == 2:
        return ((orbs.index(term[0][0]), 1), (orbs.index(term[1][0]), 0))
    if len(term) == 4:
        return ((orbs.index(term[0][0]), 1), (orbs.index(term[1][0]), 0), 
               (orbs.index(term[2][0]), 1), (orbs.index(term[3][0]), 0))   
    return None

def solve_enums(cas_tbt, k, ne_per_block = 0, ne_range = 0, balance_t = 10):
    """Solve for number of electrons in each CAS block with FCI within the block""" 
    e_nums = []
    states = []
    E_cas = 0
    for orbs in k:
        s = orbs[0]
        t = orbs[-1] + 1
        norbs = len(orbs)
        ne = min(ne_per_block + random.randint(-ne_range, ne_range), norbs - 1)
        print(f"Ne within current block: {ne}")
#         Construct (Ne^-ne)^2 terms in matrix, to enforce structure of states
        if ne_per_block != 0:
            balance_tbt = np.zeros([norbs, norbs,  norbs, norbs,])
            for p, q in product(range(norbs), repeat = 2):
                balance_tbt[p,p,q,q] += 1
            for p in range(len(orbs)):
                balance_tbt[p,p,p,p] -= 2 * ne
#             Construct 2e tensor to enforce the Ne in the ground state.
            strength = balance_t * (1 + random.random())
#             tmp_tbt = np.add(tmp_tbt, balance_tbt)
        flag = True
        while flag:
            balance_tbt *= strength
            cas_tbt[s:t, s:t, s:t, s:t] = np.add(cas_tbt[s:t, s:t, s:t, s:t], balance_tbt)
            tmp = feru.get_ferm_op(cas_tbt[s:t, s:t, s:t, s:t], True)
            sparse_H_tmp = of.get_sparse_operator(tmp)
            tmp_E_min, t_sol = of.get_ground_state(sparse_H_tmp)
            st = sdstate(n_qubit = len(orbs))
            for i in range(len(t_sol)):
                if np.linalg.norm(t_sol[i]) > np.finfo(np.float32).eps:
                    st += sdstate(s = i, coeff = t_sol[i])
#             print(f"state norm: {st.norm()}")
            st.normalize()
            E_st = st.exp(tmp)
            flag = False
            for sd in st.dic:
                ne_computed = bin(sd)[2:].count('1')
                if ne_computed != ne:
#                     print("Not enough balance, adding more terms")
                    flag = True
                    break
#             flag = False
        print(f"E_min: {tmp_E_min} for orbs: {orbs}")
        print(f"current state Energy: {E_st}")
        E_cas += E_st
        states.append(st)
        e_nums.append(ne)                
    return e_nums, states, E_cas

# Killer Construction
def construct_killer(k, e_num, n = 0, const = 1e-2, t = 1e2, n_killer = 3):
    """ Construct a killer operator for CAS Hamiltonian, based on cas block structure of k and the size of killer is 
    given in k, the number of electrons in each CAS block of the ground state
    is specified by e_nums. t is the strength of quadratic balancing terms for the killer with respect to k,
    n_killer specifies the number of operators O to choose.
    """
    if not n:
        n = max([max(orbs) for orbs in k])
    killer = of.FermionOperator.zero()
    for i in range(len(k)):
        orbs = k[i]
        outside_orbs = [j for j in range(n) if j not in orbs]
    #     Construct Ne
        Ne = sum([of.FermionOperator("{}^ {}".format(i, i)) for i in orbs])
    #     Construct O, for O as combination of Epq which preserves Sz and S2
        if len(outside_orbs) >= 4:
            tmp = 0
            while tmp < n_killer:
                p, q = random.sample(outside_orbs, 2)
                if abs(p - q) > 1:
#                     Constructing symmetry conserved killers
                    O = of.FermionOperator.zero()
                    if p % 2 != 0:
                        p -= 1
                    if q % 2 != 0:
                        q -= 1
                    ferm_op = of.FermionOperator("{}^ {}".format(p, q)) + of.FermionOperator("{}^ {}".format(q, p))
                    O += ferm_op
                    O += of.hermitian_conjugated(ferm_op)
                    ferm_op = of.FermionOperator("{}^ {}".format(p + 1, q + 1)) + of.FermionOperator("{}^ {}".format(q + 1, p + 1))
                    O += ferm_op
                    O += of.hermitian_conjugated(ferm_op)
                    killer += (1 + np.random.rand()) * const * O * (Ne - e_nums[i])
                    tmp += 1
        killer += t * (1 + np.random.rand()) * const * ((Ne - e_nums[i]) ** 2)
    return killer

def construct_orbs(key: str):
#     Contruct k from the given key
    count = 0
    lis = key.split("-")
    k = []
    for i in lis:
        tmp = int(i)
        k.append(list(range(count, count + tmp)))
        count += tmp
    return k

def construct_Hamiltonian_with_solution(path, file_name,dic_key):
    with open(path + file_name, 'rb') as handle:
        dic_temp = pickle.load(handle)
        dic = dic_temp[dic_key]

    print(dic.keys())
    cas_x = dic["cas_x"]
    killer = dic["killer"]
    k = dic["k"]
    upnum = dic["upnum"]
    spin_orbs = dic["spin_orbs"]
#     CAS 2e tensor
    tbt = csau.get_cas_matrix(cas_x, spin_orbs, k)
#     Construct hidden 2e tensor and Hamiltonian
    killer_tbt = feru.get_chemist_tbt(killer, spin_orbs, spin_orb = True)
    killer_one_body = of.normal_ordered(killer - feru.get_ferm_op(killer_tbt, spin_orb=True))
    killer_onebody_matrix = feru.get_obt(killer_one_body, n = spin_orbs, spin_orb = True)
    killer_onebody_tbt = feru.onebody_to_twobody(killer_onebody_matrix)
    Htbt_killer = np.add(Htbt, killer_onebody_tbt) 
    Htbt_with_killer = np.add(tbt, Htbt_killer)
#     Set up random unitary to hide 2e tensor
    # random_uparams = np.random.rand(upnum)
    U = construct_random_sz_unitary(spin_orbs)
#     Hide 2e etensor with random unitary transformation
    Htbt_hidden = csau.cartan_orbtransf(Htbt_with_killer, U, complex = False)
#     Hidden 2e tensor without killer
    tbt_hidden = csau.cartan_orbtransf(tbt, U, complex = False)
    return tbt, tbt_hidden, Htbt_with_killer, Htbt_hidden

In [27]:
print(os. getcwd())

/Users/rick/planted_solutions/planted_solutions/CAS_Cropping


In [28]:
"""
Tensor check functions
"""

def check_for_incorrect_spin_terms(tbt_to_check):
    num_incorrect_terms = 0
    #Check no incorrect spin terms present
    num_spin_orbitals = tbt_to_check.shape[0]
    no_incorrect_terms = True
    for piter in range(num_spin_orbitals):
        for qiter in range(num_spin_orbitals):
            for riter in range(num_spin_orbitals):
                for siter in range(num_spin_orbitals):
                    if piter % 2 == 0 and qiter % 2 == 0 and riter % 2 == 0 and siter % 2 == 0:
                        continue
                    if piter % 2 == 1 and qiter % 2 == 1 and riter % 2 == 1 and siter % 2 == 1:
                        continue
                    if piter % 2 == 0 and qiter % 2 == 0 and riter % 2 == 1 and siter % 2 == 1:
                        continue
                    if piter % 2 == 1 and qiter % 2 == 1 and riter % 2 == 0 and siter % 2 == 0:
                        continue
                    if not np.isclose(tbt_to_check[piter, qiter, riter, siter], 0.0):
                        # print(f"Incorrect spin term present in two body tensor at indices {piter}, {qiter}, {riter}, {siter}: {tbt_to_check[piter, qiter, riter, siter]}")
                        no_incorrect_terms = False
                        num_incorrect_terms += 1

    return no_incorrect_terms, num_incorrect_terms

def check_tbt(tbt_to_check):
    num_spin_orbitals = tbt_to_check.shape[0]
    dummy_obt = np.zeros((num_spin_orbitals, num_spin_orbitals))
    no_incorrect_terms, num_incorrect_terms = check_for_incorrect_spin_terms(tbt_to_check)
    print(f"No incorrect spin terms present: {no_incorrect_terms}")
    print(f"Number of incorrect terms: {num_incorrect_terms}")

    spin_symm_check_passed = dmrghandler.dmrg_calc_prepare.check_spin_symmetry(one_body_tensor=dummy_obt, two_body_tensor=tbt_to_check)
    print(f"Spin symmetry check passed: {spin_symm_check_passed}")

    permutation_symmetries_complex_orbitals_check_passed = dmrghandler.dmrg_calc_prepare.check_permutation_symmetries_complex_orbitals(
        dummy_obt, tbt_to_check
    )
    print(f"Permutation symmetries complex orbitals check passed: {permutation_symmetries_complex_orbitals_check_passed}")

    permutation_symmetries_real_orbitals_check_passed = dmrghandler.dmrg_calc_prepare.check_permutation_symmetries_real_orbitals(
        dummy_obt, tbt_to_check
    )
    print(f"Permutation symmetries real orbitals check passed: {permutation_symmetries_real_orbitals_check_passed}")

In [29]:
"""
Tensor Conversion functions
"""
def chem_to_physicist_notation_spin_orbitals(tbt_local):
    # Assume tbt_local is defined such that
    # H = γ_PQRS a†_P a_Q a†_R a_S
    # PQRS are spin-orbitals

    # Output tensors assume H = H_PQ a†_P a_Q + 0.5*G_PQRS a†_P a†_R a_S a_Q

    num_spin_orbitals = tbt_local.shape[0]
    num_ortbitals = num_spin_orbitals // 2

    G_PQRS = 2*tbt_local.copy()

    H_PQ = np.zeros((num_spin_orbitals, num_spin_orbitals))
    for p in range(num_spin_orbitals):
        for q in range(num_spin_orbitals):
            for r in range(num_spin_orbitals):
                H_PQ[p, q] += 0.5*G_PQRS[p, r, r, q]

    return H_PQ, G_PQRS

def chem_spin_orb_to_phys_spatial_orb(tbt_local):
    # Assume tbt_local is defined such that
    # H = γ_PQRS a†_P a_Q a†_R a_S
    # PQRS are spin-orbitals

    # Output tensors assume H = H_ij a†_i a_j + 0.5*G_ijkl a†_i a†_k a_l a_j, 
    # `ijkl` refer to *spatial* orbitals


    #First, convert to physicist notation with spin-orbitals
    H_PQ, G_PQRS = chem_to_physicist_notation_spin_orbitals(tbt_local)

    
    (H_ij, 
    G_ijkl, 
    spin_symm_broken) = dmrghandler.dmrg_calc_prepare.spinorbitals_to_orbitals(one_body_tensor=H_PQ, 
                                                                                two_body_tensor=G_PQRS)
    print(f"Spin symmetry broken: {spin_symm_broken}")                                                                                
    return H_ij, G_ijkl


## Step 1. Load FCIDUMP file for a catalyst

In [30]:

(
one_body_tensor,
two_body_tensor,
nuc_rep_energy,
num_orbitals,
num_spin_orbitals,
num_electrons,
two_S,
two_Sz,
orb_sym,
extra_attributes,
) = dmrghandler.dmrg_calc_prepare.load_tensors_from_fcidump(data_file_path=Path(fcidump_path)/Path(fcidump_filename), molpro_orbsym_convention=True)

spin_orbs = 2*num_orbitals
spatial_orbs = num_orbitals

print(f"Number of spin orbitals: {spin_orbs}")

h_ij = one_body_tensor
g_ijkl = two_body_tensor
# Hamiltonian assumed is `H = E_0 + h_ij a†_i a_j + 0.5*g_ijkl a†_i a†_k a_l a_j`, `ijkl` refer to *spatial* orbitals

Parsing fcidumps_catalysts/fcidump.2_co2_6-311++G__
Number of spin orbitals: 12


## Step 2. Combine one body tensor `h_ij` with two body tensor `g_ijkl` to get combined two body tensor `Γ_pqrs,στ`

In [31]:
one_body_tensor_corrected = h_ij - 0.5*np.einsum("prrq->pq",g_ijkl) 

Λ_αα = feru.onebody_to_twobody(one_body_tensor_corrected)
Λ_ββ = Λ_αα
g_pqrs = g_ijkl

# Γ_pqrs,στ:
Γ_pqrsαα  = 2 * Λ_αα + g_pqrs
Γ_pqrsββ = 2 * Λ_ββ + g_pqrs
Γ_pqrsαβ = g_pqrs
Γ_pqrsβα = g_pqrs

#`H = 0.5*Γ_pqrs,στ a†_pσ a_qσ a†_rτ a_sτ = Γ_PQRS a†_P a_Q a†_R a_S`, 
# `PQRS` refer to *spin* orbitals



## Step 3. Symmetrize combined two body tensor `γ_pqrs,στ`

In [32]:

γ_pqrs = 0.5*(Γ_pqrsαα  + Γ_pqrsββ + Γ_pqrsαβ + Γ_pqrsβα) / 4

# Eliminate small values
γ_pqrs[np.abs(γ_pqrs) < tol2] = 0
print(np.min(np.abs(γ_pqrs)))

γ_pqrsστ = dmrghandler.pyscf_wrappers.two_body_tensor_orbital_to_spin_orbital(γ_pqrs)
# Hamiltonian assumed is `H = γ_pqrs,στ a†_pσ a_qσ a†_rτ a_sτ = γ_PQRS a†_P a_Q a†_R a_S`, `PQRS` refer to *spin* orbitals


0.0


In [33]:
# Verify properties of γ_pqrsστ
check_tbt(γ_pqrsστ)

No incorrect spin terms present: True
Number of incorrect terms: 0
Spin symmetry check passed: True
Permutation symmetries complex orbitals check passed: True
Permutation symmetries real orbitals check passed: True


## Step 4. Obtain CAS-cropped planted solution two body tensor `tbt_1`

In [34]:
Htbt = γ_pqrsστ

k = construct_blocks(block_size, spin_orbs)
print(f"orbital splliting: {k}")
upnum, casnum, pnum = csau.get_param_num(spin_orbs, k, complex = False)

cas_tbt, cas_x = get_truncated_cas_tbt(Htbt, k, casnum)
tbt_1_orig = copy.deepcopy(cas_tbt)


orbital splliting: [[0, 1, 2, 3, 4, 5], [6, 7, 8, 9, 10, 11]]


In [35]:
# Verify properties of tbt_1
check_tbt(tbt_1_orig)

No incorrect spin terms present: True
Number of incorrect terms: 0
Spin symmetry check passed: True
Permutation symmetries complex orbitals check passed: True
Permutation symmetries real orbitals check passed: True


## Step 5. Add balance operators (controls number of electrons in each block) to get `tbt_2`

In [36]:

# cas_tbt_tmp = copy.deepcopy(cas_tbt)
e_nums, states, E_cas = solve_enums(cas_tbt, k, ne_per_block = ne_per_block,
                                    ne_range = ne_range, balance_t = balance_strength)
# assert np.allclose(cas_tbt_tmp, cas_tbt), "changed"
print(f"e_nums:{e_nums}")
print(f"E_cas: {E_cas}")
sd_sol = sdstate()
tbt_2 = copy.deepcopy(cas_tbt)



Ne within current block: 4
E_min: -83.01812288016492 for orbs: [0, 1, 2, 3, 4, 5]
current state Energy: -83.01812288016467
Ne within current block: 4
E_min: -73.26716319673902 for orbs: [6, 7, 8, 9, 10, 11]
current state Energy: -73.26716319673895
e_nums:[4, 4]
E_cas: -156.28528607690362


In [37]:
# Verify properties of tbt_2
check_tbt(tbt_2)

No incorrect spin terms present: True
Number of incorrect terms: 0
Spin symmetry check passed: True
Permutation symmetries complex orbitals check passed: True
Permutation symmetries real orbitals check passed: True


In [38]:

# The following code segment checks the state energy for the full Hamiltonian, takes exponential space 
# and time with respect to the number of blocks
#     E_sol = sd_sol.exp(cas_tbt)
#     print(f"Double check ground state energy: {E_sol}")

# Checking ground state with FCI
# Warning: This takes exponential time to run
#     Checking H_cas symmetries
if check_symmetry or FCI:
    H_cas = feru.get_ferm_op(cas_tbt, True)
    # print(f"Hamiltonian: {H_cas}")
if check_symmetry:
    Sz = of.hamiltonians.sz_operator(spatial_orbs)
    S2 = of.hamiltonians.s_squared_operator(spatial_orbs)
    # print(f"Sz: {Sz}")
    # print(f"S2: {S2}")
    assert of.FermionOperator.zero() == of.normal_ordered(of.commutator(Sz, H_cas)), "Sz symmetry broken"
    assert of.FermionOperator.zero() == of.normal_ordered(of.commutator(S2, H_cas)), f"S2 symmetry broken: {of.normal_ordered(of.commutator(S2, H_cas))}"

if FCI:
    E_min, sol = of.get_ground_state(of.get_sparse_operator(H_cas))

    E_min2, sol2 = of.get_ground_state(of.get_number_preserving_sparse_operator(
                                    fermion_op=H_cas,
                                    num_qubits=spin_orbs,
                                    num_electrons=sum(e_nums),
                                    spin_preserving=True,
                                    reference_determinant=None,
                                    excitation_level=None
                                    ))
    FCI_energy_for_comparison = E_min
    print(f"FCI Energy: {E_min}")
    print(f"FCI Energy, specified electrons, spin preserving (usually singlet): {E_min2}")
    tmp_st = sdstate(n_qubit = spin_orbs)
    for s in range(len(sol)):
        if sol[s] > np.finfo(np.float32).eps:
            tmp_st += sdstate(s, sol[s])
    #         print(bin(s))
    print(tmp_st.norm())
    tmp_st.normalize()
    print(tmp_st.exp(H_cas))


FCI Energy: -156.2852860769045
FCI Energy, specified electrons, spin preserving (usually singlet): -156.2852860769034
(0.3835245793489647+0j)
-153.07526805716253


## Step 6. Generate killer operator `K`

In [39]:
print(e_nums)

[4, 4]


In [40]:

cas_killer = construct_killer(k, e_nums, n = spin_orbs, n_killer = n_killer)


In [41]:
#Checks of cas_killer
if check_symmetry:
    assert of.FermionOperator.zero() == of.normal_ordered(of.commutator(Sz, cas_killer)), "Killer broke Sz symmetry"
    assert of.FermionOperator.zero() == of.normal_ordered(of.commutator(S2, cas_killer)), "S2 symmetry broken"

# Checking: if FCI of killer gives same result. Warning; takes exponential time 
if FCI:
    sparse_with_killer = of.get_sparse_operator(cas_killer + H_cas)
    killer_Emin, killer_sol = of.get_ground_state(sparse_with_killer)
    print(f"FCI Energy solution with killer: {killer_Emin}")

    sparse_with_killer2 = of.get_number_preserving_sparse_operator(
                                fermion_op=(cas_killer + H_cas),
                                num_qubits=spin_orbs,
                                num_electrons=sum(e_nums),
                                spin_preserving=True,
                                reference_determinant=None,
                                excitation_level=None
                                )
    killer_Emin2, killer_sol2 = of.get_ground_state(sparse_with_killer2)
    print(f"FCI Energy solution with killer, specified electrons, spin preserving (usually singlet): {killer_Emin2}")

    sd_Emin = sd_sol.exp(cas_tbt) + sd_sol.exp(cas_killer)
    print(f"sd_Emin: {sd_Emin} ")
    print(f"difference with CAS energy: {sd_Emin - killer_Emin}")

# Checking: if killer does not change ground state
    #         killer_error = sd_sol.exp(cas_killer)
    #         print(f"Solution Energy shift by killer: {killer_error}")
        #     killer_E_sol = sd_sol.exp(H_cas + cas_killer)
        #     print(f"Solution Energy with killer: {killer_E_sol}")


FCI Energy solution with killer: -156.2852860769032
FCI Energy solution with killer, specified electrons, spin preserving (usually singlet): -156.2852860769035
sd_Emin: 0 
difference with CAS energy: 156.2852860769032


## Step 7. Save planted solution to `.pkl` file

In [42]:

planted_sol = {}
planted_sol["E_min"] = E_cas
planted_sol["e_nums"] = e_nums
planted_sol["sol"] = sd_sol
planted_sol["killer"] = cas_killer
planted_sol["cas_x"] = cas_x
planted_sol["k"] = k
planted_sol["casnum"] = casnum
planted_sol["pnum"] = pnum
planted_sol["upnum"] = upnum
planted_sol["spin_orbs"] = spin_orbs
# print(planted_sol)
ps_path = "planted_solutions/"
f_name = fcidump_filename.split(".")[1] + ".pkl"
print(ps_path +f_name)

l = list(map(len, k))
l = list(map(str, l))
key = "-".join(l)
print(key)
if os.path.exists(ps_path + f_name):
    with open(ps_path + f_name, 'rb') as handle:
        dic = pickle.load(handle)
else:
    dic = {}

with open(ps_path + f_name, 'wb') as handle:
    dic[key] = planted_sol
    pickle.dump(dic, handle, protocol=pickle.HIGHEST_PROTOCOL)




planted_solutions/2_co2_6-311++G__.pkl
6-6


## Step 8. Use `construct_Hamiltonian_with_solution` to load `.pkl` file and produce `tbt_1`, `tbt_1_hidden`, `tbt_3`, `tbt_3_hidden`

In [43]:
tbt_1, tbt_1_hidden, tbt_3, tbt_3_hidden = construct_Hamiltonian_with_solution(
    path="planted_solutions/", file_name="2_co2_6-311++G__.pkl", dic_key="6-6"
)

dict_keys(['E_min', 'e_nums', 'sol', 'killer', 'cas_x', 'k', 'casnum', 'pnum', 'upnum', 'spin_orbs'])


In [44]:
# Verify properties of tbt_1
check_tbt(tbt_1)

No incorrect spin terms present: True
Number of incorrect terms: 0
Spin symmetry check passed: True
Permutation symmetries complex orbitals check passed: True
Permutation symmetries real orbitals check passed: True


In [45]:
# Verify properties of tbt_1_hidden
check_tbt(tbt_1_hidden)

No incorrect spin terms present: True
Number of incorrect terms: 0
Spin symmetry check passed: True
Permutation symmetries complex orbitals check passed: True
Permutation symmetries real orbitals check passed: True


In [145]:
# Verify properties of tbt_3
check_tbt(tbt_3)

No incorrect spin terms present: False
Number of incorrect terms: 15500
Spin symmetry check passed: True
Permutation symmetries complex orbitals check passed: True
Permutation symmetries real orbitals check passed: True


In [146]:
# Verify properties of tbt_3_hidden
check_tbt(tbt_3_hidden)

No incorrect spin terms present: False
Number of incorrect terms: 28812
Spin symmetry check passed: True
Permutation symmetries complex orbitals check passed: True
Permutation symmetries real orbitals check passed: True


In [147]:
# Demonstrate tbt_1_orig and tbt_1 are the same
print(np.max(np.abs(tbt_1_orig-tbt_1)))

0.0


In [148]:
# Demonstrate tbt_1 and tbt_2 are different
print(np.max(np.abs(tbt_1-tbt_2)))

120.33451575015317


### Step 8b - Show DMRG results match FCI for spin-orbital case with tbt_2

In [46]:
if FCI:
    obt_to_use, tbt_to_use = chem_to_physicist_notation_spin_orbitals(tbt_2)
    num_spin_orbitals = tbt_to_use.shape[0]
    num_orbitals = num_spin_orbitals // 2
    num_electrons = sum(e_nums)
    num_unpaired_electrons = 0
    # multiplicity = num_unpaired_electrons + 1
    nuc_rep_energy=0
    init_state_bond_dimension= 100
    max_num_sweeps=20
    
    print(f"# of electrons: {sum(e_nums)}")
    print(f"Shape of obt: {obt_to_use.shape}")

    sweep_schedule_bond_dims = [init_state_bond_dimension] * 4 + [
        init_state_bond_dimension
    ] * 4
    sweep_schedule_noise = [1e-4] * 4 + [1e-5] * 4 + [0]
    sweep_schedule_davidson_threshold = [1e-10] * 8
    energy_convergence_threshold = 1e-8

    dmrg_parameters = {
                "factor_half_convention": True,
                "symmetry_type": "SGF",
                "num_threads": 1,
                "n_mkl_threads": 1,
                "num_orbitals": 2*num_orbitals, # Extra 2 necessary for SGF
                "num_spin_orbitals": 2*num_spin_orbitals, # Extra 2 necessary for SGF
                "num_electrons": num_electrons,
                "two_S": num_unpaired_electrons,
                "two_Sz": num_unpaired_electrons,
                "orb_sym": np.zeros(num_orbitals,dtype=np.int_).tolist(),
                "temp_dir": "./tests/temp",
                "stack_mem": 1073741824,
                "restart_dir": "./tests/restart",
                "core_energy": nuc_rep_energy,
                "reordering_method": "none",
                "init_state_seed": 64241,  # 0 means random seed
                "initial_mps_method": "random",
                "init_state_bond_dimension": init_state_bond_dimension,
                "occupancy_hint": None,
                "full_fci_space_bool": True,
                "init_state_direct_two_site_construction_bool": False,
                "max_num_sweeps": max_num_sweeps,
                "energy_convergence_threshold": energy_convergence_threshold,
                "sweep_schedule_bond_dims": sweep_schedule_bond_dims,
                "sweep_schedule_noise": sweep_schedule_noise,
                "sweep_schedule_davidson_threshold": sweep_schedule_davidson_threshold,
                "davidson_type": None,  # Default is None, for "Normal"
                "eigenvalue_cutoff": 1e-20,  # Cutoff of eigenvalues, default is 1e-20
                "davidson_max_iterations": 4000,  # Default is 4000
                "davidson_max_krylov_subspace_size": 50,  # Default is 50
                "lowmem_noise_bool": False,  # Whether to use a lower memory version of the noise, default is False
                "sweep_start": 0,  # Default is 0, where to start sweep
                "initial_sweep_direction": None,  # Default is None, True means forward sweep (left-to-right)
                "stack_mem_ratio": 0.8,  # Default is 0.4
            }
    dmrg_results_dict = dmrghandler.qchem_dmrg_calc.single_qchem_dmrg_calc(
        one_body_tensor=obt_to_use,
        two_body_tensor=tbt_to_use,
        dmrg_parameters=dmrg_parameters,
        verbosity=1,
    )

    dmrg_ground_state_energy = dmrg_results_dict["dmrg_ground_state_energy"]

# of electrons: 8
Shape of obt: (12, 12)
integral cutoff error =  0.0
mpo terms =        222

Build MPO | Nsites =    12 | Nterms =        222 | Algorithm = FastBIP | Cutoff = 1.00e-20
 Site =     0 /    12 .. Mmpo =     8 DW = 0.00e+00 NNZ =        8 SPT = 0.0000 Tmvc = 0.000 T = 0.001
 Site =     1 /    12 .. Mmpo =    20 DW = 0.00e+00 NNZ =       20 SPT = 0.8750 Tmvc = 0.000 T = 0.001
 Site =     2 /    12 .. Mmpo =    19 DW = 0.00e+00 NNZ =       44 SPT = 0.8842 Tmvc = 0.000 T = 0.001
 Site =     3 /    12 .. Mmpo =    12 DW = 0.00e+00 NNZ =       28 SPT = 0.8772 Tmvc = 0.000 T = 0.001
 Site =     4 /    12 .. Mmpo =     7 DW = 0.00e+00 NNZ =       12 SPT = 0.8571 Tmvc = 0.000 T = 0.001
 Site =     5 /    12 .. Mmpo =     2 DW = 0.00e+00 NNZ =        6 SPT = 0.5714 Tmvc = 0.000 T = 0.000
 Site =     6 /    12 .. Mmpo =     8 DW = 0.00e+00 NNZ =        5 SPT = 0.6875 Tmvc = 0.000 T = 0.001
 Site =     7 /    12 .. Mmpo =    22 DW = 0.00e+00 NNZ =       20 SPT = 0.8864 Tmvc = 0.000 T

In [47]:
print(f"# unpaired electrons: {num_unpaired_electrons}")

# unpaired electrons: 0


In [48]:
# Compare FCI and DMRG energies
if FCI:
    print("Note: tbt_2 is tbt_1 plus the balance operators, and not returned by construct_Hamiltonian_with_solution")
    print(f"FCI Energy, tbt_2: {FCI_energy_for_comparison}")
    print(f"DMRG Energy, tbt_2: {dmrg_ground_state_energy}")
    print(f"Energy difference: {FCI_energy_for_comparison - dmrg_ground_state_energy}")

Note: tbt_2 is tbt_1 plus the balance operators, and not returned by construct_Hamiltonian_with_solution
FCI Energy, tbt_2: -156.2852860769045
DMRG Energy, tbt_2: -156.28528607689995
Energy difference: -4.547473508864641e-12


## Step 9. Convert tensors from spin orbitals to spatial orbitals

In [49]:
tbt_1_H_ij, tbt_1_G_ijkl = chem_spin_orb_to_phys_spatial_orb(tbt_1)

Spin symmetry broken: False


In [50]:
tbt_1_hidden_H_ij, tbt_1_hidden_G_ijkl = chem_spin_orb_to_phys_spatial_orb(tbt_1_hidden)

One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body

In [35]:
tbt_2_H_ij, tbt_2_G_ijkl = chem_spin_orb_to_phys_spatial_orb(tbt_2)

Spin symmetry broken: False


In [36]:
tbt_3_H_ij, tbt_3_G_ijkl = chem_spin_orb_to_phys_spatial_orb(tbt_3)

Spin symmetry broken: False


In [37]:
tbt_3_hidden_H_ij, tbt_3_hidden_G_ijkl = chem_spin_orb_to_phys_spatial_orb(tbt_3_hidden)

One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body symmetry check failed.
One body

## Step 10. Save `H_ij` and `G_ijkl` to FCIDUMP file

In [181]:
# Make dir
fcidump_output_path = Path(fcidump_output_path)
fcidump_output_path.mkdir(parents=True, exist_ok=True)

In [182]:
label = "_tbt_1"
filename = fcidump_filename + f"{label}"
pyscf.tools.fcidump.from_integrals(
    fcidump_output_path/Path(filename),
    tbt_1_H_ij,
    tbt_1_G_ijkl,
    nmo=tbt_1_G_ijkl.shape[0],
    nelec=np.sum(e_nums),
    nuc=0,
    # ms=0, 
    ms=1,
    orbsym=None,
    tol=1E-8,
    float_format=' %.16g',
)

In [183]:
label = "_tbt_1_hidden"
filename = fcidump_filename + f"{label}"
pyscf.tools.fcidump.from_integrals(
    fcidump_output_path/Path(filename),
    tbt_1_hidden_H_ij,
    tbt_1_hidden_G_ijkl,
    nmo=tbt_1_G_ijkl.shape[0],
    nelec=np.sum(e_nums),
    nuc=0,
    # ms=0,
    ms=1,
    orbsym=None,
    tol=1E-8,
    float_format=' %.16g',
)

In [254]:
one_body_tensor, two_body_tensor = chem_to_physicist_notation_spin_orbitals(tbt_1_hidden)
num_spin_orbitals = one_body_tensor.shape[0]
num_orbitals = num_spin_orbitals // 2
for piter in range(num_orbitals):
    for qiter in range(num_orbitals):
        if not (
            np.allclose(
                one_body_tensor[2 * piter, 2 * qiter],
                one_body_tensor[2 * piter + 1, 2 * qiter + 1],
            )
        ):
            print("not passing")

In [51]:
if FCI:
    obt_to_use, tbt_to_use = chem_to_physicist_notation_spin_orbitals(tbt_1)
    num_spin_orbitals = tbt_to_use.shape[0]
    num_orbitals = num_spin_orbitals // 2
    num_electrons = sum(e_nums)
    num_unpaired_electrons = 0
    # multiplicity = num_unpaired_electrons + 1
    nuc_rep_energy=0
    init_state_bond_dimension= 100
    max_num_sweeps=20
    
    print(f"# of electrons: {sum(e_nums)}")
    print(f"Shape of obt: {obt_to_use.shape}")

    sweep_schedule_bond_dims = [init_state_bond_dimension] * 4 + [
        init_state_bond_dimension
    ] * 4
    sweep_schedule_noise = [1e-4] * 4 + [1e-5] * 4 + [0]
    sweep_schedule_davidson_threshold = [1e-10] * 8
    energy_convergence_threshold = 1e-8

    dmrg_parameters = {
                "factor_half_convention": True,
                "symmetry_type": "SGF",
                "num_threads": 1,
                "n_mkl_threads": 1,
                "num_orbitals": 2*num_orbitals, # Extra 2 necessary for SGF
                "num_spin_orbitals": 2*num_spin_orbitals, # Extra 2 necessary for SGF
                "num_electrons": num_electrons,
                "two_S": num_unpaired_electrons,
                "two_Sz": num_unpaired_electrons,
                "orb_sym": np.zeros(num_orbitals,dtype=np.int_).tolist(),
                "temp_dir": "./tests/temp",
                "stack_mem": 1073741824,
                "restart_dir": "./tests/restart",
                "core_energy": nuc_rep_energy,
                "reordering_method": "none",
                "init_state_seed": 64241,  # 0 means random seed
                "initial_mps_method": "random",
                "init_state_bond_dimension": init_state_bond_dimension,
                "occupancy_hint": None,
                "full_fci_space_bool": True,
                "init_state_direct_two_site_construction_bool": False,
                "max_num_sweeps": max_num_sweeps,
                "energy_convergence_threshold": energy_convergence_threshold,
                "sweep_schedule_bond_dims": sweep_schedule_bond_dims,
                "sweep_schedule_noise": sweep_schedule_noise,
                "sweep_schedule_davidson_threshold": sweep_schedule_davidson_threshold,
                "davidson_type": None,  # Default is None, for "Normal"
                "eigenvalue_cutoff": 1e-20,  # Cutoff of eigenvalues, default is 1e-20
                "davidson_max_iterations": 4000,  # Default is 4000
                "davidson_max_krylov_subspace_size": 50,  # Default is 50
                "lowmem_noise_bool": False,  # Whether to use a lower memory version of the noise, default is False
                "sweep_start": 0,  # Default is 0, where to start sweep
                "initial_sweep_direction": None,  # Default is None, True means forward sweep (left-to-right)
                "stack_mem_ratio": 0.8,  # Default is 0.4
            }
    dmrg_results_dict = dmrghandler.qchem_dmrg_calc.single_qchem_dmrg_calc(
        one_body_tensor=obt_to_use,
        two_body_tensor=tbt_to_use,
        dmrg_parameters=dmrg_parameters,
        verbosity=1,
    )

    dmrg_ground_state_energy = dmrg_results_dict["dmrg_ground_state_energy"]

# of electrons: 8
Shape of obt: (12, 12)
integral cutoff error =  0.0
mpo terms =        222

Build MPO | Nsites =    12 | Nterms =        222 | Algorithm = FastBIP | Cutoff = 1.00e-20
 Site =     0 /    12 .. Mmpo =     8 DW = 0.00e+00 NNZ =        8 SPT = 0.0000 Tmvc = 0.000 T = 0.001
 Site =     1 /    12 .. Mmpo =    20 DW = 0.00e+00 NNZ =       20 SPT = 0.8750 Tmvc = 0.000 T = 0.000
 Site =     2 /    12 .. Mmpo =    19 DW = 0.00e+00 NNZ =       44 SPT = 0.8842 Tmvc = 0.000 T = 0.001
 Site =     3 /    12 .. Mmpo =    12 DW = 0.00e+00 NNZ =       28 SPT = 0.8772 Tmvc = 0.000 T = 0.000
 Site =     4 /    12 .. Mmpo =     7 DW = 0.00e+00 NNZ =       12 SPT = 0.8571 Tmvc = 0.000 T = 0.001
 Site =     5 /    12 .. Mmpo =     2 DW = 0.00e+00 NNZ =        6 SPT = 0.5714 Tmvc = 0.000 T = 0.000
 Site =     6 /    12 .. Mmpo =     8 DW = 0.00e+00 NNZ =        5 SPT = 0.6875 Tmvc = 0.000 T = 0.000
 Site =     7 /    12 .. Mmpo =    22 DW = 0.00e+00 NNZ =       20 SPT = 0.8864 Tmvc = 0.000 T

In [ ]:
# Commented out as spin symmetry broken 
# label = "tbt_1_hidden"
# filename = fcidump_filename + f"{label}"
# pyscf.tools.fcidump.from_integrals(
#     fcidump_output_path/Path(filename),
#     tbt_1_hidden_H_ij,
#     tbt_1_hidden_G_ijkl,
#     nmo=tbt_1_hidden_H_ij.shape[0],
#     nelec=np.sum(e_nums),
#     nuc=0,
#     ms=0,
#     orbsym=None,
#     tol=1E-8,
#     float_format=' %.16g',
# )

In [34]:
# Commented out as spin symmetry broken 
# label = "tbt_2"
# filename = fcidump_filename + f"{label}"
# pyscf.tools.fcidump.from_integrals(
#     fcidump_output_path/Path(filename),
#     tbt_2_H_ij,
#     tbt_2_G_ijkl,
#     nmo=tbt_2_H_ij.shape[0],
#     nelec=np.sum(e_nums),
#     nuc=0,
#     ms=0,
#     orbsym=None,
#     tol=1E-8,
#     float_format=' %.16g',
# )

In [35]:
# Commented out as spin symmetry broken 
# label = "tbt_3"
# filename = fcidump_filename + f"{label}"
# pyscf.tools.fcidump.from_integrals(
#     fcidump_output_path/Path(filename),
#     tbt_3_H_ij,
#     tbt_3_G_ijkl,
#     nmo=tbt_3_H_ij.shape[0],
#     nelec=np.sum(e_nums),
#     nuc=0,
#     ms=0,
#     orbsym=None,
#     tol=1E-8,
#     float_format=' %.16g',
# )

In [36]:
# Commented out as spin symmetry broken 
# label = "tbt_3_hidden"
# filename = fcidump_filename + f"{label}"
# pyscf.tools.fcidump.from_integrals(
#     fcidump_output_path/Path(filename),
#     tbt_3_hidden_H_ij,
#     tbt_3_hidden_G_ijkl,
#     nmo=tbt_3_hidden_H_ij.shape[0],
#     nelec=np.sum(e_nums),
#     nuc=0,
#     ms=0,
#     orbsym=None,
#     tol=1E-8,
#     float_format=' %.16g',
# )

## Step 11. Load FCIDUMP file for tbt_1

In [37]:
label = "_tbt_1"
filename = fcidump_filename + f"{label}"
    

(
one_body_tensor_dmrg,
two_body_tensor_dmrg,
nuc_rep_energy_dmrg,
num_orbitals_dmrg,
num_spin_orbitals_dmrg,
num_electrons_dmrg,
two_S_dmrg,
two_Sz_dmrg,
orb_sym_dmrg,
extra_attributes_dmrg,
) = dmrghandler.dmrg_calc_prepare.load_tensors_from_fcidump(data_file_path=fcidump_output_path/Path(filename), molpro_orbsym_convention=True)


Parsing fcidumps_catalysts_planted_solutions/fcidump.2_co2_6-311++G___tbt_1


## Step 12. Run DMRG in 'SU(2)' symmetry mode

In [38]:
obt_to_use = one_body_tensor_dmrg
tbt_to_use = two_body_tensor_dmrg

init_state_bond_dimension= 100
max_num_sweeps=20

sweep_schedule_bond_dims = [init_state_bond_dimension//2] * 4 + [
    init_state_bond_dimension
] * 4
sweep_schedule_noise = [1e-4] * 4 + [1e-5] * 4 + [0]
sweep_schedule_davidson_threshold = [1e-10] * 8
energy_convergence_threshold = 1e-8

dmrg_parameters = {
            "factor_half_convention": True,
            "symmetry_type": "SU(2)",
            "num_threads": 1,
            "n_mkl_threads": 1,
            "num_orbitals": num_orbitals_dmrg,
            "num_spin_orbitals": num_spin_orbitals_dmrg,
            "num_electrons": num_electrons_dmrg,
            "two_S": two_S_dmrg,
            "two_Sz": two_Sz_dmrg,
            "orb_sym": orb_sym_dmrg,
            "temp_dir": "./tests/temp",
            "stack_mem": 1073741824,
            "restart_dir": "./tests/restart",
            "core_energy": nuc_rep_energy_dmrg,
            "reordering_method": "none",
            "init_state_seed": 98416,#64241,  # 0 means random seed
            "initial_mps_method": "random",
            "init_state_bond_dimension": init_state_bond_dimension,
            "occupancy_hint": None,
            "full_fci_space_bool": True,
            "init_state_direct_two_site_construction_bool": False,
            "max_num_sweeps": max_num_sweeps,
            "energy_convergence_threshold": energy_convergence_threshold,
            "sweep_schedule_bond_dims": sweep_schedule_bond_dims,
            "sweep_schedule_noise": sweep_schedule_noise,
            "sweep_schedule_davidson_threshold": sweep_schedule_davidson_threshold,
            "davidson_type": None,  # Default is None, for "Normal"
            "eigenvalue_cutoff": 1e-20,  # Cutoff of eigenvalues, default is 1e-20
            "davidson_max_iterations": 4000,  # Default is 4000
            "davidson_max_krylov_subspace_size": 50,  # Default is 50
            "lowmem_noise_bool": False,  # Whether to use a lower memory version of the noise, default is False
            "sweep_start": 0,  # Default is 0, where to start sweep
            "initial_sweep_direction": None,  # Default is None, True means forward sweep (left-to-right)
            "stack_mem_ratio": 0.8,  # Default is 0.4
        }
dmrg_results_dict = dmrghandler.qchem_dmrg_calc.single_qchem_dmrg_calc(
    one_body_tensor=obt_to_use,
    two_body_tensor=tbt_to_use,
    dmrg_parameters=dmrg_parameters,
    verbosity=1,
)
dmrg_ground_state_energy_tbt_1 = dmrg_results_dict["dmrg_ground_state_energy"]

integral cutoff error =  0.0
mpo terms =        114

Build MPO | Nsites =     6 | Nterms =        114 | Algorithm = FastBIP | Cutoff = 1.00e-20
 Site =     0 /     6 .. Mmpo =    12 DW = 0.00e+00 NNZ =       12 SPT = 0.0000 Tmvc = 0.000 T = 0.002
 Site =     1 /     6 .. Mmpo =     8 DW = 0.00e+00 NNZ =       23 SPT = 0.7604 Tmvc = 0.000 T = 0.001
 Site =     2 /     6 .. Mmpo =     2 DW = 0.00e+00 NNZ =        7 SPT = 0.5625 Tmvc = 0.000 T = 0.002
 Site =     3 /     6 .. Mmpo =    13 DW = 0.00e+00 NNZ =       12 SPT = 0.5385 Tmvc = 0.000 T = 0.002
 Site =     4 /     6 .. Mmpo =    14 DW = 0.00e+00 NNZ =       51 SPT = 0.7198 Tmvc = 0.000 T = 0.002
 Site =     5 /     6 .. Mmpo =     1 DW = 0.00e+00 NNZ =       14 SPT = 0.0000 Tmvc = 0.000 T = 0.006
Ttotal =      0.014 Tmvc-total = 0.000 MPO bond dimension =    14 MaxDW = 0.00e+00
NNZ =          119 SIZE =          346 SPT = 0.6561

Rank =     0 Ttotal =      0.028 MPO method = FastBipartite bond dimension =      14 NNZ =          11

In [39]:
# Compare FCI and DMRG energies
if FCI:
    print("Below comparison only valid if balance_strength == 0")
    print(f"Balance strength: {balance_strength}")
    print(f"FCI Energy: {FCI_energy_for_comparison}")
    print(f"DMRG Energy, tbt_1: {dmrg_ground_state_energy_tbt_1}")
    print(f"Energy difference: {FCI_energy_for_comparison - dmrg_ground_state_energy_tbt_1}")

Below comparison only valid if balance_strength == 0
Balance strength: 2
FCI Energy: -128.66849127100852
DMRG Energy, tbt_1: -29.33595854711037
Energy difference: -99.33253272389815
